In [12]:
import numpy as np
import keras
from keras.datasets import cifar10
from keras.layers import Dense, Conv2D, Flatten, Activation, MaxPooling2D, Dropout
from keras.models import Sequential
from keras.utils import np_utils


In [13]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
n_examples = 5000

In [14]:
X1_train = []
X1_test = []
X2_train = []
X2_test = []
Y1_train = []
Y1_test = []
Y2_train = []
Y2_test = []

for ix in range(n_examples):
    if y_train[ix] < 5:
        # put data in set 1
        X1_train.append(x_train[ix]/255.0)
        Y1_train.append(y_train[ix])
    else:
        # put data in set 2
        X2_train.append(x_train[ix]/255.0)
        Y2_train.append(y_train[ix])

for ix in range(y_test.shape[0]):
    if y_test[ix] < 5:
        # put data in set 1
        X1_test.append(x_test[ix]/255.0)
        Y1_test.append(y_test[ix])
    else:
        # put data in set 2
        X2_test.append(x_test[ix]/255.0)
        Y2_test.append(y_test[ix])


In [15]:
#Convert the input to an array
X1_train = np.asarray(X1_train).reshape((-1, 32, 32, 3))
X1_test = np.asarray(X1_test).reshape((-1, 32, 32, 3))
X2_train = np.asarray(X2_train).reshape((-1, 32, 32, 3))
X2_test = np.asarray(X2_test).reshape((-1, 32, 32, 3))

# One hot representation for multiclass problem
# Convert 1-dimensional class arrays to 10-dimensional class matrices
Y1_train = np_utils.to_categorical(np.asarray(Y1_train), num_classes=5)
Y1_test = np_utils.to_categorical(np.asarray(Y1_test), num_classes=5)

Y2_train = np_utils.to_categorical(np.asarray(Y2_train), num_classes=10)
Y2_test = np_utils.to_categorical(np.asarray(Y2_test), num_classes=10)

In [16]:
print (X1_train.shape, X1_test.shape)
print (Y1_train.shape, Y1_test.shape)


(2489, 32, 32, 3) (5000, 32, 32, 3)
(2489, 5) (5000, 5)


In [17]:
split1 = int(0.8 * X1_train.shape[0])
split2 = int(0.8 * X2_train.shape[0])

x1_val = X1_train[split1:]
x1_train = X1_train[:split1]
y1_val = Y1_train[split1:]
y1_train = Y1_train[:split1]

x2_val = X2_train[split2:]
x2_train = X2_train[:split2]
y2_val = Y2_train[split2:]
y2_train = Y2_train[:split2]

http://cs231n.github.io/assets/conv-demo/index.html

In [18]:
model = Sequential()

model.add(Conv2D(32, (5, 5), input_shape=(32, 32, 3), activation='relu'))
model.add(Conv2D(16, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(8, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dropout(0.42))

model.add(Dense(128, activation='relu'))
model.add(Dense(5,activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 32)        2432      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 16)        12816     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 10, 8)         1160      
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 800)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               102528    
__________

In [19]:
import time, datetime

start = datetime.datetime.now()
hist1 = model.fit(x1_train, y1_train,
                  batch_size=100,
                  epochs=10,
                  verbose=2,
                  shuffle=True)

time_taken = datetime.datetime.now() - start
print ('\n'*2, '-'*20, '\n')
print ('Time taken for first training: ', time_taken)
print ('\n', '-'*20, '\n'*2)

Epoch 1/10
7s - loss: 1.5662 - acc: 0.2597
Epoch 2/10
7s - loss: 1.3770 - acc: 0.4083
Epoch 3/10
6s - loss: 1.2199 - acc: 0.4756
Epoch 4/10
6s - loss: 1.1540 - acc: 0.5063
Epoch 5/10
6s - loss: 1.0935 - acc: 0.5439
Epoch 6/10
6s - loss: 1.0372 - acc: 0.5595
Epoch 7/10
6s - loss: 0.9731 - acc: 0.5937
Epoch 8/10
6s - loss: 0.9697 - acc: 0.5937
Epoch 9/10
6s - loss: 0.9336 - acc: 0.6128
Epoch 10/10
6s - loss: 0.9277 - acc: 0.6193


 -------------------- 

Time taken for first training:  0:01:05.858178

 -------------------- 




In [20]:
for l in model.layers[:6]:
    l.trainable = False  

In [21]:
trans_model = Sequential(model.layers[:6])

trans_model.add(Dense(128))
trans_model.add(Activation('relu'))
trans_model.add(Dense(10))
trans_model.add(Activation('softmax'))

trans_model.summary()
trans_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 32)        2432      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 16)        12816     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 10, 8)         1160      
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 800)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               102528    
__________

In [22]:
start = datetime.datetime.now()
hist2 = trans_model.fit(x2_train, y2_train, epochs=10, shuffle=True, batch_size=100, verbose=2)
time_taken = datetime.datetime.now() - start
print ('\n'*2, '-'*20, '\n')
print ('Time taken for final training: ', time_taken)
print ('\n', '-'*20, '\n'*2)

Epoch 1/10
3s - loss: 1.5387 - acc: 0.4293
Epoch 2/10
2s - loss: 1.0460 - acc: 0.5926
Epoch 3/10
3s - loss: 0.9592 - acc: 0.6290
Epoch 4/10
3s - loss: 0.9216 - acc: 0.6429
Epoch 5/10
3s - loss: 0.8818 - acc: 0.6718
Epoch 6/10
2s - loss: 0.8738 - acc: 0.6569
Epoch 7/10
2s - loss: 0.8625 - acc: 0.6668
Epoch 8/10
2s - loss: 0.8459 - acc: 0.6798
Epoch 9/10
2s - loss: 0.8368 - acc: 0.6783
Epoch 10/10
2s - loss: 0.8012 - acc: 0.7007


 -------------------- 

Time taken for final training:  0:00:30.993987

 -------------------- 


